In [ ]:
import base64
import io
import json
import uuid

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import widgets, Tab
from IPython.display import display, clear_output, Javascript

In [ ]:
EXIT_FILE = "../data/test_exits.json"
ROOM_FILE = "../data/test_rooms.json"

In [ ]:
# Load data function
def load_json_data(filename):
    with open(filename, 'r', encoding="utf-8") as file:
        return json.load(file)

# Global variables to store data
rooms_data = load_json_data(ROOM_FILE)
exits_data = load_json_data(EXIT_FILE)

In [ ]:
def create_graph():
    G = nx.MultiDiGraph()
    
    # Create a set of all valid exit IDs
    valid_exit_ids = set()
    for room in rooms_data['rooms']:
        valid_exit_ids.update(room.get('ExitID', []))
    
    # Add nodes (rooms) to the graph
    for room in rooms_data['rooms']:
        G.add_node(room['RoomID'], title=room['Title'])
    
    # Create a map of exit IDs to exit data
    exit_map = {exit['ExitID']: exit for exit in exits_data['exits'] if exit['ExitID'] in valid_exit_ids}
    
    # Add edges (exits) to the graph
    for room in rooms_data['rooms']:
        for exit_id in room.get('ExitID', []):
            exit = exit_map.get(exit_id)
            if exit:
                G.add_edge(
                    room['RoomID'],
                    exit['TargetRoom'],
                    key=exit_id,
                    direction=exit['Direction']
                )
    
    return G

In [ ]:
# Visualization function
def visualize_graph(G):
    fig, ax = plt.subplots(figsize=(14, 10))
    pos = nx.spring_layout(G, k=1.5, iterations=200)
    
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=700, ax=ax)
    node_labels = {node: G.nodes[node]['title'] for node in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=10, ax=ax)
    
    edge_list = list(G.edges(keys=True, data=True))
    edge_counts = {}
    
    for u, v, key, data in edge_list:
        edge_counts[(u, v)] = edge_counts.get((u, v), 0) + 1
    
    for (u, v), count in edge_counts.items():
        edges = [(u, v2, k, d) for (u, v2, k, d) in edge_list if v2 == v]
        num_edges = len(edges)
        rad_list = [0.0] if num_edges == 1 else np.linspace(-0.5, 0.5, num_edges)
        
        for (u, v, key, data), rad in zip(edges, rad_list):
            nx.draw_networkx_edges(
                G, pos,
                edgelist=[(u, v, key)],
                connectionstyle=f'arc3,rad={rad}',
                arrowsize=15,
                edge_color='gray',
                ax=ax
            )
            
            x1, y1 = pos[u]
            x2, y2 = pos[v]
            xm, ym = (x1 + x2) / 2, (y1 + y2) / 2
            
            dx, dy = x2 - x1, y2 - y1
            angle = np.arctan2(dy, dx)
            
            horizontal_scale = 0.05
            vertical_offset = rad * 0.1
            
            label_x = xm - horizontal_scale * np.sin(angle)
            label_y = ym + vertical_offset * np.cos(angle)
            
            label = data.get('direction', 'No Label')
            ax.text(
                label_x,
                label_y,
                label,
                fontsize=8,
                ha='center',
                va='center',
                bbox=dict(facecolor='white', edgecolor='none', pad=1)
            )
    
    ax.set_title("Room Layout Graph")
    ax.axis('off')
    
    # Convert plot to image
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)
    
    return img_str


In [ ]:
def create_exit_editor_widgets():
    exit_list = widgets.Select(
        options=[(f"{exit['ExitID']}: {exit['Direction']} to {exit['TargetRoom']}", exit['ExitID']) for exit in exits_data['exits']],
        description='Exits:',
        layout=widgets.Layout(width='300px')
    )

    exit_id = widgets.Text(description='Exit ID:', disabled=True)
    direction = widgets.Text(description='Direction:')
    target_room = widgets.IntText(description='Target Room:')
    visible = widgets.Checkbox(description='Visible', value=True)

    new_button = widgets.Button(description='New Exit')
    save_button = widgets.Button(description='Save Exit')
    delete_button = widgets.Button(description='Delete Exit')
    save_all_button = widgets.Button(description='Save All Exits')

    output = widgets.Output()

    return exit_list, exit_id, direction, target_room, visible, new_button, save_button, delete_button, save_all_button, output


In [ ]:
def on_exit_select(change, exit_id, direction, target_room, visible):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_exit = next((exit for exit in exits_data['exits'] if exit['ExitID'] == change['new']), None)
        if selected_exit:
            exit_id.value = selected_exit['ExitID']
            direction.value = selected_exit['Direction']
            target_room.value = selected_exit['TargetRoom']
            visible.value = selected_exit['Visible']

In [ ]:
def on_new_exit(b, exit_list, exit_id, direction, target_room, visible):
    new_exit = {
        'ExitID': str(uuid.uuid4()),
        'Direction': '',
        'TargetRoom': 0,
        'Visible': True
    }
    exits_data['exits'].append(new_exit)
    
    # Find the room that this exit belongs to
    for room in rooms_data['rooms']:
        if room['RoomID'] == target_room.value:
            room['ExitID'].append(new_exit['ExitID'])
            break
    else:
        print(f"Warning: Room with ID {target_room.value} not found. Exit not added to any room.")
    
    update_exit_list(exit_list)
    exit_list.value = new_exit['ExitID']
    on_exit_select({'type': 'change', 'name': 'value', 'new': new_exit['ExitID']}, exit_id, direction, target_room, visible)

In [ ]:
def on_save_exit(b, exit_list, exit_id, direction, target_room, visible, output):
    selected_exit = next((exit for exit in exits_data['exits'] if exit['ExitID'] == exit_id.value), None)
    if selected_exit:
        selected_exit['Direction'] = direction.value
        selected_exit['TargetRoom'] = target_room.value
        selected_exit['Visible'] = visible.value
        update_exit_list(exit_list)
        with output:
            clear_output()
            print(f"Exit {exit_id.value} updated successfully.")

In [ ]:
def remove_exit_from_rooms(exit_id):
    for room in rooms_data['rooms']:
        if exit_id in room['ExitID']:
            room['ExitID'].remove(exit_id)

In [ ]:
def on_delete_exit(b, exit_list, exit_id, direction, target_room, visible, output):
    exit_to_delete = next((exit for exit in exits_data['exits'] if exit['ExitID'] == exit_id.value), None)
    if exit_to_delete:
        exits_data['exits'] = [exit for exit in exits_data['exits'] if exit['ExitID'] != exit_id.value]
        
        # Remove the exit from rooms_data
        for room in rooms_data['rooms']:
            if exit_id.value in room['ExitID']:
                room['ExitID'].remove(exit_id.value)
        
        update_exit_list(exit_list)
        if exits_data['exits']:
            exit_list.value = exits_data['exits'][0]['ExitID']
            on_exit_select({'type': 'change', 'name': 'value', 'new': exit_list.value}, exit_id, direction, target_room, visible)
        else:
            clear_exit_fields(exit_id, direction, target_room, visible)
        with output:
            clear_output()
            print(f"Exit {exit_id.value} deleted successfully.")
    else:
        with output:
            clear_output()
            print(f"Exit {exit_id.value} not found.")

In [ ]:
def on_save_all(b, output):
    with open(EXIT_FILE, 'w') as f:
        json.dump(exits_data, f, indent=2)
    with open(ROOM_FILE, 'w') as f:
        json.dump(rooms_data, f, indent=2)
    with output:
        clear_output()
        print("All exits and rooms saved successfully.")

In [ ]:
def update_exit_list(exit_list):
    exit_list.options = [(f"{exit['ExitID']}: {exit['Direction']} to {exit['TargetRoom']}", exit['ExitID']) for exit in exits_data['exits']]

In [ ]:
def clear_exit_fields(exit_id, direction, target_room, visible):
    exit_id.value = ''
    direction.value = ''
    target_room.value = 0
    visible.value = True

In [ ]:
def exit_app(b):
    display(Javascript('window.parent.postMessage({"jupyter_voila_exit": true}, "*")'))

In [ ]:
def create_room_visualization_tab():
    output = widgets.Output()
    image = widgets.Image()
    update_button = widgets.Button(description="Update Graph")
    exit_button = widgets.Button(description="Exit Application", button_style='danger')
    
    def update_graph(b):
        G = create_graph()  # Recreate the graph
        with output:
            clear_output()
        img_str = visualize_graph(G)
        image.value = base64.b64decode(img_str)
    
    update_button.on_click(update_graph)
    exit_button.on_click(exit_app)
    
    room_viz_tab = widgets.VBox([
        widgets.HBox([update_button, exit_button]),
        image,
        output
    ])
    
    update_graph(None)  # Initial graph update
    return room_viz_tab

In [ ]:
def create_exit_editor_tab():
    exit_list, exit_id, direction, target_room, visible, new_button, save_button, delete_button, save_all_button, output = create_exit_editor_widgets()

    exit_list.observe(lambda change: on_exit_select(change, exit_id, direction, target_room, visible), names='value')
    new_button.on_click(lambda b: on_new_exit(b, exit_list, exit_id, direction, target_room, visible))
    save_button.on_click(lambda b: on_save_exit(b, exit_list, exit_id, direction, target_room, visible, output))
    delete_button.on_click(lambda b: on_delete_exit(b, exit_list, exit_id, direction, target_room, visible, output))
    save_all_button.on_click(lambda b: on_save_all(b, output))

    exit_app_button = widgets.Button(description="Exit Application", button_style='danger')
    exit_app_button.on_click(exit_app)

    return widgets.VBox([
        widgets.HBox([exit_list, widgets.VBox([exit_id, direction, target_room, visible])]),
        widgets.HBox([new_button, save_button, delete_button, save_all_button, exit_app_button]),
        output
    ])

In [ ]:
# Main application layout
def room_editor_app():
    room_viz_tab = create_room_visualization_tab()
    exit_editor_tab = create_exit_editor_tab()
    
    tab = Tab(children=[room_viz_tab, exit_editor_tab])
    tab.set_title(0, 'Room Visualization')
    tab.set_title(1, 'Exit Editor')
    
    def on_tab_change(change):
        if change['new'] == 0:  # Room Visualization tab
            room_viz_tab.children[0].children[0].click()  # Simulate click on update button
    
    tab.observe(on_tab_change, names='selected_index')
    
    app_layout = widgets.VBox([
        widgets.HTML("<h1>Room Editor</h1>"),
        tab
    ])
    
    return app_layout

In [ ]:
# Display the app
display(room_editor_app())